<a href="https://colab.research.google.com/github/Mihir1703/Deep-Learning/blob/master/GTSRB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q kaggle

from google.colab import files
files.upload()

!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets list

Saving kaggle.json to kaggle.json
ref                                                             title                                           size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------------------  ---------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
meirnizri/covid19-dataset                                       COVID-19 Dataset                                 5MB  2022-11-13 15:47:17          14099        403  1.0              
thedevastator/analyzing-credit-card-spending-habits-in-india    Credit Card Spending Habits in India           319KB  2022-12-14 07:30:37           1116         44  1.0              
michals22/coffee-dataset                                        Coffee dataset                                  24KB  2022-12-15 20:02:12           3264         75  1.0              
thedevastator/unlock-profits-with-e-commerce-sales-

In [2]:
import os

if os.path.isdir('dataset') is False:
    os.makedirs('dataset')

In [3]:
!kaggle datasets download -d meowmeowmeowmeowmeow/gtsrb-german-traffic-sign
!unzip -q gtsrb-german-traffic-sign.zip -d dataset

100% 609M/612M [00:30<00:00, 22.3MB/s]
100% 612M/612M [00:30<00:00, 21.1MB/s]


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from keras.layers import Dense
from keras.models import Sequential
import keras.utils as image
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

In [5]:
classes = os.listdir('dataset/Train')
classes[:5]

['10', '13', '4', '27', '36']

In [10]:
train_batch=ImageDataGenerator().flow_from_directory(directory='dataset/Train',target_size=(224,224),classes=(os.listdir('dataset/Train')),batch_size=100)

Found 39209 images belonging to 43 classes.


In [11]:
model = tf.keras.Sequential([
    keras.layers.Conv2D(filters=16, kernel_size=(3,3), activation='relu'),
    keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu'), 
    keras.layers.MaxPool2D(pool_size=(2, 2)), 
    keras.layers.BatchNormalization(axis=1),  
    keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu'), 
    keras.layers.Conv2D(filters=128, kernel_size=(3,3), activation='relu'), 
    keras.layers.MaxPool2D(pool_size=(2, 2)), 
    keras.layers.BatchNormalization(axis=-1),  
    keras.layers.Flatten(), 
    keras.layers.Dense(512, activation='relu'), 
    keras.layers.BatchNormalization(), 
    keras.layers.Dropout(rate=0.5),  
    keras.layers.Dense(43, activation='softmax')
])
model.build([None, 224, 224, 3])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 222, 222, 16)      448       
                                                                 
 conv2d_5 (Conv2D)           (None, 220, 220, 32)      4640      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 110, 110, 32)     0         
 2D)                                                             
                                                                 
 batch_normalization_3 (Batc  (None, 110, 110, 32)     440       
 hNormalization)                                                 
                                                                 
 conv2d_6 (Conv2D)           (None, 108, 108, 64)      18496     
                                                                 
 conv2d_7 (Conv2D)           (None, 106, 106, 128)    

In [12]:
opt = Adam(learning_rate=0.001, decay=0.001 / (10 * 0.5)) 
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy']) 

In [ ]:
with tf.device('/gpu:0'):
    model.fit(x=train_batch,epochs=10)

Epoch 1/10
279/393 [====================>.........] - ETA: 50s - loss: 0.9982 - accuracy: 0.7386

In [ ]:
model.save('gtsrb.h5')

In [ ]:
test_data = pd.read_csv('dataset/Test.csv')
test_data.head()

In [ ]:
def process_img(file):
    img = image.load_img(file,target_size=(224,224))
    return np.array(img) / 255

In [ ]:
test_samples = []
test_labels = []

for i in test_data.to_numpy():
    test_labels.append(i[6])
    test_samples.append(process_img('dataset/' + i[7]))
test_samples = np.array(test_samples)
test_labels = np.array(test_labels)


In [ ]:
predict = model.predict(x=test_samples)
predict